In [6]:
from langsmith import Client
import os
client = Client()
project_name=""
def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")
_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("LANGCHAIN_API_YI_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.environ["LANGCHAIN_API_YI_KEY"]

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "xmode-vqa-gpt_4o-english-100-with-intent"
project_name=os.environ["LANGCHAIN_PROJECT"]
project_runs = list(client.list_runs(project_name=project_name, is_root=True))

In [7]:
# Filter out only successful runs
success_project_runs = [run for run in project_runs if run.status == "success"]

In [8]:
len(success_project_runs)

100

In [9]:
success_project_runs[0].__dict__.keys()

dict_keys(['id', 'name', 'start_time', 'run_type', 'end_time', 'extra', 'error', 'serialized', 'events', 'inputs', 'outputs', 'reference_example_id', 'parent_run_id', 'tags', 'session_id', 'child_run_ids', 'child_runs', 'feedback_stats', 'app_path', 'manifest_id', 'status', 'prompt_tokens', 'completion_tokens', 'total_tokens', 'first_token_time', 'total_cost', 'prompt_cost', 'completion_cost', 'parent_run_ids', 'trace_id', 'dotted_order', 'in_dataset'])

In [10]:
from decimal import Decimal
from uuid import UUID
from datetime import datetime
from tqdm import tqdm as tqdm
from time import sleep

def handle_value(value):
    if isinstance(value, UUID):
        return str(value)
    if isinstance(value, Decimal):
        return float(value)
    if isinstance(value, datetime):
        return value.isoformat()
    if isinstance(value, dict):
        return {key: handle_value(val) for key, val in value.items()}
    if isinstance(value, list):
        return [handle_value(val) for val in value]
    return value



def _extract_run(run):
    keys = run.__dict__.keys()
    temp =  {key: handle_value(run.__dict__[key]) for key in keys}
    p_id = temp["id"]
    if isinstance(temp.get('child_run_ids', None), list) and len(temp['child_run_ids']) > 0:
        # print("****", temp['child_run_ids'])
        c_runs = []
        max_retry = 3
        retry = 0
        try:
            c_runs = list(client.list_runs(project_name=project_name, parent_run_id=p_id))
            sleep(1)
        except:
            while len(c_runs) == 0 and retry < max_retry:
                retry += 1
                try:
                    c_runs = list(client.list_runs(project_name=project_name, parent_run_id=p_id))
                    sleep(10)
                except:
                    print("retrying...")
                
        # print("add child:", [r.id for r in c_runs])
        res = {
            **temp,
            'child_runs': [_extract_run(_run) for _run in c_runs]
        }
        if not any(res['child_runs']):
            res['child_runs'] = None
        return res
    if temp.get('type','ChatGenerationChunk') == 'ChatGenerationChunk':
        return
    return temp

In [11]:
import json
from time import sleep
from tqdm import tqdm as tqdm
for run in tqdm(success_project_runs[::-1], total=len(success_project_runs)):
    run = _extract_run(run)
    with open("experiments/xmode/en/xmode-vqa-m3ae-star-100-en-gpt_4o-with-intent-langsmith-extract_all.json", "r+") as f:
        # load existing data
        try:
            data = json.load(f)
        except json.JSONDecodeError:
            data = []
        # update data
        data.append(run)
        # save data
        f.seek(0)
        json.dump(data, f, indent=2)
        f.truncate()
    sleep(1)

100%|██████████| 100/100 [2:18:18<00:00, 82.98s/it]  


In [ ]:
res = []
for run in success_project_runs:
    res.append(_extract_run(run))

In [ ]:
import json
with open("experiments/xmode/en/metadata_xmode-vqa-m3ae-star-100-en-gpt_4o-with-intent.json", "w") as f:
    json.dump(res[::-1], f, indent=2) # reverse the order to match the order of the questions in the dataset

In [38]:
with open("experiments/xmode/en/metadata_xmode-vqa-m3ae-star-100-en-gpt_4o-with-intent.json", "r") as f:
    res = json.load(f)


In [41]:
data_path = "/home/ubuntu/workspace/XMODE-LLMCompiler/experiments/xmode/en/xmode-vqa-m3ae-star-100-en-gpt_4o-with-intent.json"
with open(data_path, "r") as f:
    data = json.load(f)

for d,r in zip(data,res):
    assert d["question"]==r["question"], print(d["question"],r["question"])
    d["prompt_tokens"] = r["prompt_tokens"]
    d["completion_tokens"] = r["completion_tokens"]
    d["total_tokens"] = r["total_tokens"]
    d["prompt_cost"] = r["prompt_cost"]
    d["completion_cost"] = r["completion_cost"]
    d["total_cost"] = r["total_cost"]

merged_data_path = "/home/ubuntu/workspace/XMODE-LLMCompiler/experiments/xmode/en/xmode-vqa-m3ae-star-100-en-gpt_4o-with-intent-with-meta.json"
with open(merged_data_path, "w") as f:
    json.dump(data, f, indent=2)

import pandas as pd
df = pd.DataFrame(data)
df.to_csv("/home/ubuntu/workspace/XMODE-LLMCompiler/experiments/xmode/en/xmode-vqa-m3ae-star-100-en-gpt_4o-with-intent-with-meta.csv", index=False)


In [42]:
df.to_excel("/home/ubuntu/workspace/XMODE-LLMCompiler/experiments/xmode/en/xmode-vqa-m3ae-star-100-en-gpt_4o-with-intent-with-meta.xlsx", index=False)

"given the last study of patient 10284038 in 2105, is the cardiac silhouette's width larger than half of the total thorax width?"